In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"azazel0201","key":"258a2565f9959f5e026e5237fe107179"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d marcosgabriel/photovoltaic-system-thermography

 84% 73.0M/86.6M [00:01<00:00, 62.5MB/s]
100% 86.6M/86.6M [00:01<00:00, 75.0MB/s]


In [ ]:
!unzip /content/photovoltaic-system-thermography.zip

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!make


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2024-04-04 19:03:38--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  18.1MB/s    in 8.6s    

2024-04-04 19:03:47 (27.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights /content/dataset_1/images/001R.jpg

In [ ]:
import os
import json

def parse_json_annotations(json_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    for json_file in json_files:
        with open(os.path.join(json_dir, json_file), 'r') as f:
            data = json.load(f)

        img_name = os.path.splitext(json_file)[0] + '.txt'
        with open(os.path.join(output_dir, img_name), 'w') as f:
            for instance in data['instances']:
                x_center = instance['center']['x'] / IMAGE_WIDTH
                y_center = instance['center']['y'] / IMAGE_HEIGHT
                width = (instance['corners'][1]['x'] - instance['corners'][0]['x']) / IMAGE_WIDTH
                height = (instance['corners'][2]['y'] - instance['corners'][0]['y']) / IMAGE_HEIGHT
                class_id = 0  # Assuming only one class for now
                if instance['defected_module']:
                    class_id = 1  # Defected module class
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Define the width and height of your images
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

# Directory containing JSON annotations
json_dir = '/content/dataset_1/annotations/'

# Directory to save YOLO-formatted annotation files
output_dir = '/content/yolo_annotations/'

parse_json_annotations(json_dir, output_dir)


In [ ]:
./darknet detector train data/obj.data cfg/yolov3.cfg darknet53.conv.74 -map

SyntaxError: invalid syntax (<ipython-input-1-7912a0b241ec>, line 1)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageOps
import torchvision.models as models
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, txt_file, transform=None, target_size=416, padding_value=[0, 0, 0, 0, 2]):
        self.data = []
        self.transform = transform
        self.target_size = target_size
        self.padding_value = padding_value

        with open(txt_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip().split()
                image_path = line[0]
                bboxes = [list(map(float, line[i:i+4])) for i in range(1, len(line), 5)]
                self.data.append((image_path, bboxes))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, bboxes = self.data[idx]
        image = Image.open(image_path).convert("RGB")

        # Resize image maintaining aspect ratio
        width, height = image.size
        aspect_ratio = width / height
        if aspect_ratio > 1:
            new_width = self.target_size
            new_height = int(self.target_size / aspect_ratio)
        else:
            new_height = self.target_size
            new_width = int(self.target_size * aspect_ratio)

        image = image.resize((new_width, new_height))

        # Pad image to target size if necessary
        pad_width = max(0, self.target_size - new_width)
        pad_height = max(0, self.target_size - new_height)
        if pad_width > 0 or pad_height > 0:  # Pad if either pad_width or pad_height is greater than 0
            pad_color = tuple(self.padding_value[:3])
            padding = Image.new(image.mode, (self.target_size, self.target_size), pad_color)
            padding.paste(image, ((self.target_size - new_width) // 2, (self.target_size - new_height) // 2))
            image = padding

        # Convert image to tensor
        image = transforms.ToTensor()(image)

        # Apply custom transformation to mask padded regions
        mask = torch.all(image == torch.tensor(self.padding_value[:3]).view(3, 1, 1), dim=0)
        mask = mask.float()
        image = image * (1 - mask)

        # Apply normalization
        image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)

        return image, bboxes


class MaskPaddedRegions(object):
    def __init__(self, padding_value):
        self.padding_value = padding_value

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # Calculate mask to identify padded regions
        mask = torch.all(image == self.padding_value, dim=0)
        # Convert mask to dtype=torch.float32 for multiplication
        mask = mask.float()
        # Apply mask to the image to zero out padded regions
        image = image * (1 - mask)
        return {'image': image, 'label': label}

In [ ]:
import os
import json

dataset_folder = 'dataset_1'
images_folder = os.path.join(dataset_folder, 'images')
annotations_folder = os.path.join(dataset_folder, 'annotations')
output_file = 'dataset.txt'  # File to save dataset information

# Define your classes
classes = ['defected', 'not defected']

# Initialize the dataset list
dataset = []

# Iterate through annotation files
for annotation_file in os.listdir(annotations_folder):
    if annotation_file.endswith('.json'):
        image_name = os.path.splitext(annotation_file)[0]
        image_path = os.path.join(images_folder, image_name + '.jpg')  # Assuming images are in JPEG format

        # Load JSON file
        with open(os.path.join(annotations_folder, annotation_file), 'r') as f:
            data = json.load(f)

        # Extract bounding box annotations for both classes
        annotations = []
        for instance in data['instances']:
            if instance['defected_module']:
                class_id = classes.index('defected')
            else:
                class_id = classes.index('not defected')

            corners = instance['corners']
            x_min = min(corners, key=lambda x: x['x'])['x']
            x_max = max(corners, key=lambda x: x['x'])['x']
            y_min = min(corners, key=lambda x: x['y'])['y']
            y_max = max(corners, key=lambda x: x['y'])['y']

            annotations.append((x_min, y_min, x_max, y_max, class_id))

        # Add image path and annotations to the dataset
        dataset.append((image_path, annotations))

# Write dataset to file
with open(output_file, 'w') as f:
    for image_path, annotations in dataset:
        f.write(image_path)
        for annotation in annotations:
            f.write(' ')
            f.write(' '.join(str(coord) for coord in annotation))
        f.write('\n')

print("Dataset created successfully.")


In [ ]:
# Open the input file for reading
with open('dataset.txt', 'r') as infile:
    # Read all lines from the file
    lines = infile.readlines()

max_points = 0
for l in lines:
    points = (l.split(' ')[1:])
    max_points = max(len(points), max_points)


for i, l in enumerate(lines):
    points = (l.split(' ')[1:])
    if len(points) != max_points:
        add = (max_points - len(points)) // 5  # Calculate the number of sets of (0 0 0 0 2) to add
        padding = ' '.join(['0', '0', '0', '0', '2'] * add)
        lines[i] = lines[i].rstrip() + ' ' + padding + '\n'  # Add padding and newline


with open('dataset2.txt', 'w') as outfile:
    outfile.writelines(lines)
exit()
# Determine the maximum number of data points in any row
max_data_points = max(len(line.split()) for line in lines)
print(max_data_points)

# Pad each row with (0 0 0 0 2) where necessary
for i, line in enumerate(lines):
    data_points = line.split()
    num_data_points = len(data_points)
    if num_data_points < max_data_points:
        # Pad the row with (0 0 0 0 2) until it reaches the maximum length
        padding = ['0', '0', '0', '0', '2'] * (max_data_points - num_data_points)
        padded_line = ' '.join(data_points + padding)
        lines[i] = padded_line + '\n'  # Add newline character

# Write the modified lines back to the file
with open('dataset2.txt', 'w') as outfile:
    outfile.writelines(lines)


In [ ]:
import os
import cv2
import json
import numpy as np
import torch
import torch.nn.functional as F

# Step 1: Prepare Dataset

# Load images and corresponding JSON files
def load_dataset(image_dir, annotation_dir):
    image_paths = []
    annotations = []

    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(image_dir, filename)
            annotation_path = os.path.join(annotation_dir, filename.replace('.jpg', '.json').replace('.png', '.json'))
            if os.path.exists(annotation_path):
                image_paths.append(image_path)
                with open(annotation_path, 'r') as f:
                    annotations.append(json.load(f)['instances'])

    return image_paths, annotations

# Step 2: Prepare YOLOv3 Model

# Define paths to YOLOv3 configuration file, pre-trained weights, and class names
yolo_cfg_path = "yolov3.cfg"
yolo_weights_path = "yolov3.weights"
class_names_path = "coco2.names"

# Update YOLOv3 configuration file for rectangle detection
def update_yolo_config_for_rectangles(yolo_cfg_path):
    with open(yolo_cfg_path, 'r') as f:
        cfg = f.read()

    # Modify last layer for rectangle detection
    cfg = cfg.replace('classes=80', 'classes=1')
    cfg = cfg.replace('filters=255', 'filters=30')  # (4+1)*6 = 30, adjust filters accordingly
    cfg = cfg.replace('activation=linear', 'activation=linear')  # linear activation for regression

    with open(yolo_cfg_path, 'w') as f:
        f.write(cfg)

# Update YOLOv3 configuration file
update_yolo_config_for_rectangles(yolo_cfg_path)

# Load YOLOv3 configuration and weights
net = cv2.dnn.readNet(yolo_weights_path, yolo_cfg_path)



# Load class names
classes = None
with open(class_names_path, 'r') as f:
    classes = f.read().strip().split('\n')

# Step 3: Load Pre-trained Weights (already done in Step 2)

# Step 4: Train the Model (fine-tuning)

def train_yolo_model(image_paths, annotations):
    # Fine-tune YOLOv3 model using images and annotations
    for i, image_path in enumerate(image_paths):
        image = cv2.imread(image_path)
        height, width, channels = image.shape
        # print(height, width, channels)
        # exit()

        # Extract rectangle annotations
        rectangles = annotations[i]


        # Define target output for each rectangle
        targets = []
        for rect in rectangles:
            corners = rect["corners"]
            center = rect["center"]

            # print(corners)
            # print(center)
            x_center = center["x"]
            y_center = center["y"]

            for corner in corners:
                x = corner['x']
                y = corner['y']
                target_x = (x + x_center) / (2 * width)
                target_y = (y + y_center) / (2 * height)
                target_width = abs(x - x_center) / width
                target_height = abs(y - y_center) / height
                targets.append([target_x, target_y, target_width, target_height])

        # Create input blob
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        print(blob)
        cv2.imshow("image", cv2.imread(image_path))
        cv2.waitKey(0)
        net.setInput(blob)

        # Forward pass
        outs = net.forward()

        print(outs)
        exit()
        # Compute loss (MSE loss or any suitable regression loss)
        loss = compute_loss(outs, targets)
        # Update model weights

# Step 5: Evaluate Performance (Optional)
def compute_loss(predictions, targets):
    # Convert predictions and targets to PyTorch tensors
    predictions = torch.tensor(predictions, requires_grad=True)
    targets = torch.tensor(targets, requires_grad=False)

    # Compute Mean Squared Error (MSE) loss
    loss = F.mse_loss(predictions, targets)

    # Backpropagation
    loss.backward()

    # Get the gradients
    gradients = predictions.grad

    # Update the model weights (for demonstration purposes)
    # Replace this with actual weight update code in your YOLOv3 implementation
    with torch.no_grad():
        learning_rate = 0.001  # Adjust as needed
        predictions -= learning_rate * gradients

    return loss.item()  # Return the loss value

def detect_rectangles(image_path):
    # Load image
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Create input blob
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Forward pass
    outs = net.forward()

    # Process output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # Draw rectangle on image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return image

# Example usage:
image_paths, annotations = load_dataset("dataset_1\\images", "dataset_1\\annotations")
# print(annotations[0])
train_yolo_model(image_paths, annotations)
# result_image = detect_rectangles("test_image.jpg")
# cv2.imshow("Result", result_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

def detect_objects(image):
    height, width, channels = image.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    return image, len(boxes)

# Read the input image
image = cv2.imread("dataset_1\\images\\001R.jpg")

# Detect objects and draw bounding boxes
result, num_objects = detect_objects(image.copy())

# Display the result
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print the number of objects detected
print("Number of objects detected:", num_objects)


In [ ]:
import cv2
import os

def overlay_images(folder_path, output_path, transparency=0.5):
    images = []

    # Iterate through the images in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            img = cv2.imread(image_path)
            images.append(img)

    # Create a blank image to overlay on
    base_image = images[0].copy()

    # Overlay each image on top of the base image
    for img in images[1:]:
        cv2.addWeighted(img, transparency, base_image, 1 - transparency, 0, base_image)

    # Show the final image
    cv2.imshow("Overlayed Image", base_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

folder_path = "dataset_1/images"
output_path = "image.png"
overlay_images(folder_path, output_path, transparency=0.5)


In [ ]:
import cv2
import numpy as np



def merge_close_coordinates(points, threshold=5):
    merged_points = []
    i = 0
    while i < len(points):
        merged_point = points[i].flatten()
        x1, y1 = merged_point

        j = i + 1
        while j < len(points):
            next_point = points[j].flatten()
            x2, y2 = next_point

            # Check if x and y values are close enough
            if abs(x2 - x1) < threshold and abs(y2 - y1) < threshold:
                # Merge the points
                merged_point[0] = (x1 + x2) // 2
                merged_point[1] = (y1 + y2) // 2
                # Move the index to the next point
                i = j
            else:
                # If points are not close enough, break the loop
                break
            j += 1

        merged_points.append(merged_point)
        i += 1

    return merged_points

def detect_rectangles(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize a counter for rectangles
    rectangle_count = 0

    # Iterate through detected contours and find rectangles
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 300:
            epsilon = 0.02 * perimeter
            approx = cv2.approxPolyDP(contour, epsilon, True)
            # Merge close coordinates
            merged_points = merge_close_coordinates(approx)
            approx = np.array(merged_points).reshape((-1, 1, 2)).astype(np.int32)
            # If the polygon has 4 vertices (a rectangle), draw a bounding box
            if len(approx) < 20000000:
                print(approx)
                x, y, w, h = cv2.boundingRect(approx)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                rectangle_count += 1

    return image, rectangle_count

# Read the input image
image = cv2.imread("dataset_1\\images\\001R.jpg")

# Detect rectangles and draw bounding boxes
result, num_rectangles = detect_rectangles(image.copy())

# Display the result
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print the number of rectangles detected
print("Number of rectangles:", num_rectangles)
